# AlexNet — Architektúra

Súbor: alexnet_2.ipynb

Program: Hospodárska informatika

Vypracovala:  Veronika Motúzová 

Bakalárska práca: Metódy hlbokého učenia pre detekciu meteorov v astronomických snímkach

Vedúci bakalárskej práce: doc. Ing. Peter Butka, PhD.

Konzultant: Ing. Viera Maslej Krešňáková

Import knižníc

In [2]:
import numpy as np
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense, Flatten
from keras.metrics import categorical_crossentropy
from keras.layers import Conv2D, MaxPooling2D, Dropout, ZeroPadding2D
from keras.utils import np_utils, plot_model
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
import array
from sklearn.metrics import confusion_matrix,classification_report
import itertools 
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from keras.callbacks import ModelCheckpoint
from keras.models import load_model 

Načítanie dát

In [3]:
train = np.load('../data/data_final_train.npz')
test = np.load('../data/data_final_test.npz')

In [4]:
x_train = train['train_data'].reshape(train['train_data'].shape[0], 128, 128, 1)
x_test = test['test_data'].reshape(test['test_data'].shape[0], 128, 128, 1)

In [5]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [6]:
print(x_train.shape)
print(x_test.shape)

(23746, 128, 128, 1)
(23746, 128, 128, 1)


In [7]:
y_train = np_utils.to_categorical(train['train_targets'], 2)
y_test = np_utils.to_categorical(test['test_targets'], 2)

Začíname vrstviť neurónku AlexNet Architecture

In [10]:
model = Sequential()

In [11]:
model.add(ZeroPadding2D((1,1),input_shape=(128, 128,1)))

In [12]:
model.add(Conv2D(96, 3, strides=2, activation='relu'))

In [13]:
model.add(MaxPooling2D(3, 2))

In [14]:
model.add(Conv2D(96, 5, strides=1, padding='same', activation='relu')) 

In [15]:
model.add(MaxPooling2D(3, 2))

In [16]:
model.add(Conv2D(256, 3, strides=1, padding='same', activation='relu'))

In [17]:
model.add(Conv2D(96, 3, strides=1, padding='same', activation='relu'))

In [18]:
model.add(MaxPooling2D(3, 2))

In [19]:
model.add(Flatten())

In [20]:
model.add(Dense(384, activation='relu')) 

In [21]:
model.add(Dropout(0.25)) 

In [22]:
model.add(Dense(384, activation='relu')) 

In [23]:
model.add(Dropout(0.25)) 

In [24]:
model.add(Dense(2, activation='softmax'))

Adam Optimization

In [25]:
adam = Adam()

In [26]:
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

Aby sa uložil model s najlepším výsledkom:

In [27]:
saved_model = "alexnet_2.hdf5"
checkpoint = ModelCheckpoint(saved_model, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

In [28]:
f = model.fit(x_train, y_train, epochs=15, batch_size=128, validation_data=(x_test, y_test), callbacks=[checkpoint])


Train on 23746 samples, validate on 23746 samples
Epoch 1/15
23746/23746 [==============================] - 914s 38ms/step - loss: 0.3862 - accuracy: 0.8131 - val_loss: 0.2110 - val_accuracy: 0.9247

Epoch 00001: val_accuracy improved from -inf to 0.92475, saving model to alexnet_2.hdf5
Epoch 2/15
23746/23746 [==============================] - 652s 27ms/step - loss: 0.2223 - accuracy: 0.9178 - val_loss: 0.1765 - val_accuracy: 0.9400

Epoch 00002: val_accuracy improved from 0.92475 to 0.93999, saving model to alexnet_2.hdf5
Epoch 3/15
23746/23746 [==============================] - 578s 24ms/step - loss: 0.1745 - accuracy: 0.9376 - val_loss: 0.1507 - val_accuracy: 0.9501

Epoch 00003: val_accuracy improved from 0.93999 to 0.95005, saving model to alexnet_2.hdf5
Epoch 4/15
23746/23746 [==============================] - 566s 24ms/step - loss: 0.1424 - accuracy: 0.9522 - val_loss: 0.1215 - val_accuracy: 0.9625

Epoch 00004: val_accuracy improved from 0.95005 to 0.96252, saving model to ale

Vyhodnotenie test loss a accuracy

Načítanie modelu

In [32]:
print("Loading model....")
model = load_model('alexnet_2.hdf5')

Loading model....


In [33]:
#na trénovacej množine 
score = model.evaluate(x_train, y_train, verbose=0) 
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.030323464764591173
Test accuracy: 0.9901878237724304


In [34]:
#na testovacej množine
score1 = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score1[0])
print('Test accuracy:', score1[1])

Test loss: 0.10279645753099233
Test accuracy: 0.9743114709854126


Predikcia

In [35]:
y_pred = model.predict(x_test)
prediction_int = np.zeros_like(y_pred)
prediction_int[y_pred > 0.5] = 1

Confusion Matrix

In [36]:
matrix = metrics.confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))

In [37]:
print(matrix)
# TN / FN
# FP / TP

[[11712   161]
 [  449 11424]]


Classification Report

In [38]:
print(classification_report(y_test, prediction_int))

              precision    recall  f1-score   support

           0       0.96      0.99      0.97     11873
           1       0.99      0.96      0.97     11873

   micro avg       0.97      0.97      0.97     23746
   macro avg       0.97      0.97      0.97     23746
weighted avg       0.97      0.97      0.97     23746
 samples avg       0.97      0.97      0.97     23746



In [39]:
model.summary() 

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_2 (ZeroPaddin (None, 130, 130, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 96)        960       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 31, 31, 96)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 31, 31, 96)        230496    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 15, 15, 96)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 256)       221440    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 15, 15, 96)       